# Introduction to Rubin Observatory

Contact author: Alex Broughton


## Introduction

**Welcome to Rubin Observatory!** The Legacy Survey of Space and Time (LSST) will produce a vast amount of data, and it will be important to be able to find, visualize, reduce, and analyze this data. Data management for LSST is handled by the Data Management (DM) Stack, which encompasses all the software for handling (middleware), processing (raw images $\rightarrow$ photometric/astrometric-ally calibrated catalogs), and science. All LSST images and data will pass thorough this data pipeline.

In this tutorial, we will learn

- Learn how data is organized on the US Data Facility (USDF)
- Learn how to access embargoed summit data
- Learn how to visualize that data
- Process an image through your first data pipeline!

## 1.0 Set Up

The primary access point for data is the Rubin Science Platform (RSP). For most commissioning integration, testing, and verification and validation (V&V) tests of the telescope system and image quality, we will be using the USDF (ultimately, most major science pipelines will happen on NERSC). 

Luckily for you, all LSST-distributed software comes already installed for you!

Environments with different versions of the DM Stack are in:

        /opt/lsst/software/stack                    (on RSP)
        /sdf/group/rubin/sw/                        (on USDF terminal)

If you want to add your own configuration to your LSST environment startup on the RSP in the
LSST iPython kernel, create a sourceable shell fragment in:

        ${HOME}/notebooks/.user_setups

and it will be sourced during kernel startup.

Find useful documentation for the software and Notebook Aspect at:
- https://pipelines.lsst.io
- https://rsp.lsst.io

**Note: if you want to do this yourself (e.g. you want to run the DM Stack on your own terminal shell), you need to install the version you want and then set it up, like so:**
```
source /sdf/group/rubin/sw/${VERSION}/loadLSST.bash
setup lsst_sitcom -t ${VERSION}
```

When you start up the RSP, selecting the "Recommended" release loads in an envirnonment with the most recent stable release of the DM Stack to your kernel. To check that it is loaded, you can run:

In [ ]:
! eups list -s | grep lsst_distrib
! eups list -s | grep ip_isr
! eups list -s | grep cp_pipe

In [ ]:
# For this tutorial, we will need:
import numpy as np
import matplotlib.pyplot as plt

from lsst.daf.butler import Butler
from lsst.obs.lsst import LsstComCam
from lsst.ip.isr import IsrTaskLSST
from lsst.pipe.tasks.visualizeVisit import VisualizeMosaicExpTask

# Camera object
camera = LsstComCam.getCamera()

# This will be a useful function for later
def getRectPatchFromBBox(bbox, color, linestyle='-'):
    return patches.Rectangle(
        (bbox.minX, bbox.minY), 
        bbox.width, bbox.height, 
        linewidth=1, 
        linestyle=linestyle,
        edgecolor=color, 
        facecolor='none',
    )

\***NOTE about workspaces on USDF**:
<br>You are given 4 personal direcotries on the system: 
- `home`: `/sdf/home/{first character}/{username}`              (25GB available)
- `rubin-user`: `/sdf/group/rubin/user/{username}`              (25TB available)
- `rubin-user-data`: `/sdf/data/rubin/user/{username}`          (1 TB available)
- `scratch`: `/sdf/scratch/users/{first character}/{username}`  (IDK I personally treat this like a blackhole)


## 2.0 Data Access Using the Butler

**Data** are stored in **repositories** as **collections** of **dataset types**, and are found by their associated **dimensions**.

The repositories can be found here: `/sdf/group/rubin/repo`

In [ ]:
# Each repository has a butler
butler = Butler("/repo/embargo")
registry = butler.registry # Contains list of available data (basically a precompiled catalog of dataset type: dimensions)


In [ ]:
# Other repositories
! ls /sdf/group/rubin/repo

There are two collections in every repository when it is first set up:
```
    {Instrument}/raw/all
    {Instrument}/calib
```
Notes:
- When data is taken, it is "ingested" into the `{Instrument}/raw/all` collection with the dataset type of `raw`. E.g. in `/repo/embargo` this happens immediately, and then after 80 hours for it to get ingested into `/repo/main`.
- Calibrations will be made and processed and stored inthe `{Instrument}/calib` collection with different dataset types for different types of calibrations (e.g. `bias`, `dark`, `flat_r`, `flat_i`, etc.).
- Anytime someone runs a pipeline on some data, it will produce other dataset types (e.g. `postISRCCD`, `calexp`, `finalized_src_table`, `diaObject`, etc.) in a new collection (under `u/{username}/your/collection/name/datatimestamp}`) that will be CHAINED to all the input collections

In [ ]:
# Anytime someone 
registry.queryCollections().__len__()

In [ ]:
What data is available?
refs = list(registry.queryDatasets(
    "raw",
    instrument="LSSTComCam",
    where="exposure.day_obs=20241024 and exposure.seq_num=55",
    collections="LSSTComCam/raw/all",
))

print(f"Found {len(refs)} references of datasetType 'raw' !")

In [ ]:
# Lets look at one of these references:
ref = refs[0]
ref

In [ ]:
# Now that we have a reference for this image, 
# we can ask the butler to go and get it for us:
exposure = butler.get('raw', dataId=ref.dataId, collections="LSSTComCam/raw/all")

### Some properties of the exposure:

In [ ]:
# Image info:
#print(camera.getNameMap())
print(exposure.getMetadata())

In [ ]:
# Where the image lives
print("Exposure")
print(type(exposure))
print(type(exposure.getImage()))
print(type(exposure.getImage().getArray()))

print("\nDetector")
detector = exposure.getDetector()
print(type(detector))
print(detector.getName())
print(detector.getId())
print(type(detector.getBBox()), detector.getBBox())

print("\nAmplifier")
amplifiers = exposure.getDetector().getAmplifiers()
amp = amplifiers[0]
print(type(amp))
print([amp.getName() for amp in amplifiers])
print( type(amp.getBBox()), amp.getBBox())

## 3.0 Visualize the image

In [ ]:
plt.title(detector.getName())
plt.imshow(exposure.image.array, origin='lower')
for amp in amplifiers:
    plt.text(
        amp.getRawDataBBox().centerX - 150.,
        amp.getRawDataBBox().centerY - 150., 
        amp.getName(),
        c="w",
    )

In [ ]:
import matplotlib.patches as patches
# Create a Rectangle patch
s = amp.getRawHorizontalPrescanBBox()
horizPrescan = patches.Rectangle((s.minX, s.minY), s.width, s.height, linewidth=1, edgecolor='r', facecolor='none')
s = amp.getRawHorizontalOverscanBBox()
horizOverscan = patches.Rectangle((s.minX, s.minY), s.width, s.height, linewidth=1, edgecolor='g', facecolor='none')
s = amp.getRawVerticalOverscanBBox()
verticalOverscan = patches.Rectangle((s.minX, s.minY), s.width, s.height, linewidth=1, edgecolor='b', facecolor='none')

s = amp.getRawBBox()
rawBBox = patches.Rectangle((s.minX, s.minY), s.width, s.height, linewidth=1, edgecolor='k', facecolor='none')
s = amp.getRawDataBBox()
rawDataBBox = patches.Rectangle((s.minX, s.minY), s.width, s.height, linewidth=1, linestyle="--",edgecolor='k', facecolor='none')

plt.imshow(exposure.image.array, origin='lower', cmap='binary')
plt.gca().add_patch(horizPrescan) # red
plt.gca().add_patch(horizOverscan) # green
plt.gca().add_patch(verticalOverscan) # blue
plt.gca().add_patch(rawBBox) # black
plt.gca().add_patch(rawDataBBox) # black-dashed


## 4.0 Run Instrument Signature Removal (ISR)

This doesn't look much like an image. Let's clean it up!

In [ ]:
%%time

# We first need to get all our exposures and calibrations:
exposures = []
linearizers = []
biases = []
darks = []
defects = []
crosstalks = []
ptcs = []

for i, ref in enumerate(refs):
    exposures.append(butler.get('raw', dataId=ref.dataId, physical_filter='pinhole', collections="LSSTComCam/raw/all"))
    linearizers.append(butler.get('linearizer', instrument='LSSTComCam', detector = exposures[i].detector.getId(), collections="LSSTComCam/calib"))
    darks.append(butler.get('dark', instrument='LSSTComCam', detector = exposures[i].detector.getId(), collections="LSSTComCam/calib"))
    biases.append(butler.get('bias', instrument='LSSTComCam', detector = exposures[i].detector.getId(), collections="LSSTComCam/calib"))
    defects.append(butler.get('defects', instrument='LSSTComCam', detector = exposures[i].detector.getId(), collections="LSSTComCam/calib"))
    crosstalks.append(butler.get('crosstalk', instrument='LSSTComCam', detector = exposures[i].detector.getId(), collections="LSSTComCam/calib"))
    ptcs.append(butler.get('ptc', instrument='LSSTComCam', detector = exposures[i].detector.getId(), collections="LSSTComCam/calib"))

In [ ]:
# Get the configurations for the task
config = IsrTaskLSST.ConfigClass()
config.doFlat = False
config.doAssembleCcd = True
config.doDeferredCharge = False
config.doBrighterFatter = False
config.doCalculateStatistics = True
config.doStandardStatistics = True
config.doBias=True
config.doDark=True
config.isrStats.doCtiStatistics = True
task = IsrTaskLSST(config=config)

In [ ]:
rs = []
for i, exposure in enumerate(exposures):
    print("Processing detector", exposure.detector.getName())
    r = task.run(
        exposure,
        bias=biases[i],
        dark=darks[i],
        defects=defects[i],
        linearizer=linearizers[i],
        crosstalk=crosstalks[i],
        ptc=ptcs[i],
    )
    rs.append(r)

r = rs[1]

The result for each exposure is contained in the `rs` list. Each `r` is a struct that contains the exposure and some other useful things like some basic statistics of the image.

In [ ]:
plt.hist(rs[0].exposure.image.array.ravel(), bins=100, histtype='step')
plt.xlabel("Signal [electron]")
plt.yscale('log')

In [ ]:
from matplotlib.colors import LogNorm, SymLogNorm, AsinhNorm
#norm = SymLogNorm(linthresh=3500, vmin=3500, vmax=4000)
norm = LogNorm(vmin=3750, vmax=4000)

plt.title(detector.getName())
plt.imshow(rs[0].exposure.image.array, origin='lower',norm=norm, cmap='binary_r')
plt.colorbar()

## 5.0 Let's put it all together!

In [ ]:
config = VisualizeMosaicExpTask().ConfigClass()
config.binning = 1
task = VisualizeMosaicExpTask(config=config)

clean_exposures = [r.exposure for r in rs]
mosaic = task.run(clean_exposures, camera = camera)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(mosaic.outputData.array, norm=norm,origin='lower', cmap='binary_r')
plt.axis('off')